In [14]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt


In [15]:
df =pd.read_csv(r'data/forestfires.csv')
df

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,aug,sun,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,aug,sun,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,aug,sun,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,aug,sat,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [21]:
months =["jan","feb","mar","apr","may","jun","jul","aug","sep","oct","nov","dec"]
weekdays=["mon","tue","wed","thu","fri","sat","sun"]
for i in range(len(df["month"])):
    for j in range(len(months)):
        if df.iloc[i]["month"] ==months[j]:
            df.at[i,"month"]=j
            break
for i in range(len(df["day"])):
    for j in range(len(months)):
        if df.iloc[i]["day"] ==weekdays[j]:
            df.at[i,"day"]=j
            break

    
    


In [22]:
df

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,2,4,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.00
1,7,4,9,1,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.00
2,7,4,9,5,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.00
3,8,6,2,4,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.00
4,8,6,2,6,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,4,3,7,6,81.6,56.7,665.6,1.9,27.8,32,2.7,0.0,6.44
513,2,4,7,6,81.6,56.7,665.6,1.9,21.9,71,5.8,0.0,54.29
514,7,4,7,6,81.6,56.7,665.6,1.9,21.2,70,6.7,0.0,11.16
515,1,4,7,5,94.4,146.0,614.7,11.3,25.6,42,4.0,0.0,0.00


In [34]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

In [29]:
size =97
state =45
train_vars,test_vars,train_results,test_results =train_test_split(df.drop(columns=["area"]),df.area,test_size=1-size/100,random_state=state) 

In [30]:
reg_line =linear_model.LinearRegression()
reg_line.fit(train_vars,train_results)

predictions =reg_line.predict(test_vars)

In [31]:
tft=[y for y in test_results]
for i in range(len(tft)):
    print(f'Prediction:{predictions[i]}--Answer:{tft[i]}')




Prediction:34.22958171295909--Answer:0.0
Prediction:17.577811494775474--Answer:0.0
Prediction:-8.80246376326153--Answer:0.0
Prediction:27.175414376337017--Answer:7.8
Prediction:22.673149164514932--Answer:0.0
Prediction:15.014459065599612--Answer:0.0
Prediction:-5.815418538787396--Answer:0.0
Prediction:4.437540211277687--Answer:4.61
Prediction:3.8842947042298572--Answer:7.4
Prediction:21.01863117909884--Answer:6.43
Prediction:-11.96068777927455--Answer:0.0
Prediction:25.581142699352284--Answer:0.0
Prediction:23.421216551208218--Answer:0.36
Prediction:-0.4413979716551619--Answer:0.0
Prediction:15.859981692671441--Answer:0.0
Prediction:18.065867674026144--Answer:0.0


In [35]:
print(mean_squared_error(tft,predictions))
print(r2_score(tft,predictions))

302.46616709000017


ValueError: continuous is not supported